In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

% matplotlib inline
# Always make it pretty.
plt.style.use('ggplot')

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk.data

In [7]:
# Load data from file
df = pd.read_csv('loan.csv')

C:\Users\yuanq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1069639,1304742,7000.0,7000.0,7000.0,60 months,15.96,170.08,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1072053,1288686,3000.0,3000.0,3000.0,36 months,18.64,109.43,E,E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1071795,1306957,5600.0,5600.0,5600.0,60 months,21.28,152.39,F,F2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1071570,1306721,5375.0,5375.0,5350.0,60 months,12.69,121.45,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Show summary stats
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,8.873790e+05,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,8.873750e+05,887379.000000,887350.000000,...,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171030e+05,21372.000000,21372.000000,21372.000000
mean,3.246513e+07,3.500182e+07,14755.264605,14741.877625,14702.464383,13.246740,436.717127,7.502759e+04,18.157039,0.314442,...,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206862e+04,0.943945,1.537058,1.977307
std,2.282734e+07,2.411335e+07,8435.455601,8429.897657,8442.106732,4.381867,244.186593,6.469830e+04,17.190626,0.862244,...,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749826e+04,1.446872,2.723339,2.874067
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.320000,15.670000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,9.206643e+06,1.087713e+07,8000.000000,8000.000000,8000.000000,9.990000,260.705000,4.500000e+04,11.910000,0.000000,...,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,3.443327e+07,3.709528e+07,13000.000000,13000.000000,13000.000000,12.990000,382.550000,6.500000e+04,17.650000,0.000000,...,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,5.490814e+07,5.847135e+07,20000.000000,20000.000000,20000.000000,16.200000,572.600000,9.000000e+04,23.950000,0.000000,...,47858.000000,87.600000,2.000000,4.000000,7771.500000,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,6.861706e+07,7.354484e+07,35000.000000,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,9999.000000,39.000000,...,878459.000000,223.300000,22.000000,43.000000,127305.000000,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [10]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [42]:
df['default'] = np.where((df['loan_status'] =='Charged Off') | (df['loan_status'] == 'Default') ,'True', 'False')

In [43]:
df.default.unique()

array(['False', 'True'], dtype=object)

In [44]:
df[['default','loan_status']]

,default,loan_status
0,False,Fully Paid
1,True,Charged Off
2,False,Fully Paid
3,False,Fully Paid
4,False,Current
5,False,Fully Paid
6,False,Current
7,False,Fully Paid
8,True,Charged Off
9,True,Charged Off


In [17]:
desc = df['desc'].values.tolist()

In [18]:
type(desc),len(desc)

(list, 887379)

In [19]:
len(df.title.unique())

63145

# Nature language Processing 

In [20]:
desc[0]

'  Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>'

In [21]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from string import punctuation

In [22]:
from nltk.stem.porter   import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet  import WordNetLemmatizer

print SnowballStemmer('english').stem('running')
print WordNetLemmatizer().lemmatize('caused')

run
caused


In [23]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000)
vectors = vectorizer.fit_transform(desc).toarray()
words = vectorizer.get_feature_names()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [26]:
len(df[df['desc'].isnull()])

761351

In [27]:
df[df['desc'].isnull()]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,default
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
11,1069908,1305008,12000.0,12000.0,12000.0,36 months,12.69,402.54,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
14,1069057,1303503,10000.0,10000.0,10000.0,36 months,10.65,325.74,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
15,1069759,1304871,1000.0,1000.0,1000.0,36 months,16.29,35.31,D,D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16,1065775,1299699,10000.0,10000.0,10000.0,36 months,15.27,347.98,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
19,1069742,1304855,9200.0,9200.0,9200.0,36 months,6.03,280.01,A,A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
23,1069700,1304810,10000.0,10000.0,10000.0,36 months,11.71,330.76,B,B3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
24,1069559,1304634,6000.0,6000.0,6000.0,36 months,11.71,198.46,B,B3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
25,1069697,1273773,15000.0,15000.0,15000.0,36 months,9.91,483.38,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [28]:
df['desc'].isnull().sum()

761351

In [29]:
df['emp_title'].isnull().sum()

51462

In [30]:
df['desc'].notnull().sum()

126028

In [31]:
df['emp_title'].notnull().sum()

835917

natural language processing with missing value

### Get the dataset without missing value

In [45]:
df_1 = df[df['emp_title'].notnull()]

In [51]:
target = df_1['default'].values

### Define your target variable (any categorical variable that may be meaningful)

In [52]:
target[:10]

array(['True', 'False', 'False', 'False', 'False', 'False', 'True',
       'False', 'False', 'True'], dtype=object)

In [53]:
emp_title = df_1['emp_title'].values.tolist()

### Let's create training dataset and test dataset

In [49]:
from sklearn.cross_validation import train_test_split

C:\Users\yuanq\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [66]:
# Split to documents_train, documents_test, target_train, target_test
emp_train, emp_test, target_train, target_test = train_test_split(
    emp_title,
    target,
    test_size = 0.4,
    random_state = 42
)

### Let's get NLP representation of the documents

In [48]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000)


In [67]:
vectors_train = vectorizer.fit_transform(emp_train).toarray()

In [58]:
words = vectorizer.get_feature_names()

In [68]:
vectors_train.shape

(501550L, 2000L)

In [69]:
vectors_test = vectorizer.fit_transform(emp_test).toarray()

In [70]:
vectors_test.shape

(334367L, 2000L)

In [35]:
vectors.shape

(835917L, 2000L)

In [36]:
words

[u'11',
 u'12',
 u'13',
 u'1st',
 u'2nd',
 u'3m',
 u'3rd',
 u'911',
 u'aaa',
 u'abbott',
 u'ac',
 u'academic',
 u'academy',
 u'accenture',
 u'access',
 u'account',
 u'accountant',
 u'accounting',
 u'accounts',
 u'acct',
 u'accts',
 u'ace',
 u'acquisition',
 u'action',
 u'active',
 u'activities',
 u'activity',
 u'actuarial',
 u'actuary',
 u'ad',
 u'adjunct',
 u'adjuster',
 u'adm',
 u'admin',
 u'administration',
 u'administrative',
 u'administrator',
 u'adminstrative',
 u'adminstrator',
 u'admission',
 u'admissions',
 u'adp',
 u'adult',
 u'advance',
 u'advanced',
 u'advantage',
 u'advertising',
 u'adviser',
 u'advisor',
 u'advisors',
 u'advisory',
 u'advocate',
 u'aerospace',
 u'aetna',
 u'affairs',
 u'ag',
 u'agency',
 u'agent',
 u'agriculture',
 u'aid',
 u'aide',
 u'air',
 u'aircraft',
 u'airline',
 u'airlines',
 u'airport',
 u'airways',
 u'alabama',
 u'alarm',
 u'alaska',
 u'allen',
 u'alliance',
 u'allied',
 u'allstate',
 u'ambassador',
 u'ambulance',
 u'america',
 u'american',
 u'am

In [37]:
def get_top_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[::-1][:n]]

In [76]:
def get_bottom_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[:n]]

In [38]:
avg = np.sum(vectors, axis=0) / np.sum(vectors > 0, axis=0)
print "top 10 by average tf-idf"
print get_top_values(avg, 10, words)

top 10 by average tf-idf
[u'longshoreman', u'mailhandler', u'nordstrom', u'postmaster', u'saic', u'usps', u'fiserv', u'hairstylist', u'accenture', u'citigroup']


In [39]:
total = np.sum(vectors, axis=0)
print "top 10 by total tf-idf"
print get_top_values(total, 10, words)

top 10 by total tf-idf
[u'manager', u'teacher', u'sales', u'director', u'supervisor', u'assistant', u'engineer', u'driver', u'analyst', u'owner']


### Classifying positive/negative review

In [71]:
# Build a Logistic Regression Classifier

from sklearn.linear_model import LogisticRegression

model_lrc = LogisticRegression()
model_lrc.fit(vectors_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [72]:
# Get score for training set
model_lrc.score(vectors_train, target_train)

0.94872495264679491

In [73]:
# Get score for training set
model_lrc.score(vectors_test, target_test)

0.94846680443943332

### What are the key features that trend to default

In [75]:
# Let's find it out by ranking
n = 20
get_top_values(model_lrc.coef_[0], n, words)

[u'nestle',
 u'head',
 u'clean',
 u'distributing',
 u'hartford',
 u'square',
 u'rigger',
 u'commissioner',
 u'hand',
 u'appraisal',
 u'frozen',
 u'pta',
 u'united',
 u'heavy',
 u'shareholder',
 u'interactive',
 u'editor',
 u'researcher',
 u'deposit',
 u'port']

What are the key features that trend not to default

In [77]:
# Let's find it out by ranking
n = 20
get_bottom_values(model_lrc.coef_[0], n, words)

[u'architect',
 u'travelers',
 u'correction',
 u'partner',
 u'providence',
 u'procurement',
 u'diagnostic',
 u'pr',
 u'vista',
 u'pilot',
 u'attendant',
 u'clerk',
 u'fund',
 u'physician',
 u'database',
 u'electronics',
 u'dept',
 u'productions',
 u'parish',
 u'representative']

In [84]:
df_1[df_1['emp_title'].str.match('nestle')]['default']

14014     False
22701     False
25113     False
89509     False
112648    False
114002    False
130680     True
132163    False
139272    False
139362    False
148441    False
162312    False
169539     True
171636     True
178780    False
184269     True
194328    False
213607    False
225204    False
Name: default, dtype: object

In [91]:
df_1[df_1['emp_title'].str.match('pilot')]['default']

45827     False
46053     False
69339     False
206501    False
242394    False
251008    False
251095    False
251648    False
255411    False
277961    False
295191    False
297850    False
310551    False
312799    False
355155    False
365281    False
378237    False
384854    False
389715    False
391327    False
415994    False
418840    False
418914    False
422368    False
440749    False
486185    False
493914    False
516266    False
516806    False
523292    False
          ...  
581861    False
591365    False
600605    False
621361    False
643764    False
653099    False
670604    False
686273    False
687843    False
700434    False
700670    False
708565    False
712278    False
727047    False
737077    False
746143    False
750926    False
750982    False
757111    False
768392    False
772623    False
790463    False
803724    False
803900    False
805639    False
818766    False
824938    False
836870    False
861376    False
871073    False
Name: default, Length: 6